In [3]:

import functions_framework
import time
import random
from EmbeddingPredictionClient import EmbeddingPredictionClient  
from google.cloud import bigquery
import json
import asyncio
from datetime import datetime
import base64
import pandas as pd

async def exponential_backoff_retries(client, text=None, image_file: bytes=None, max_retries=5, embedding_type=None):
    """
    This function applies exponential backoff with retries to the API calls.
    """
    attempt = 0
    while attempt < max_retries:
        try:
            # Try to get the embedding from the client
            if embedding_type=="multimodal_embedding":                   
                    return client.get_multimodal_embedding("", image_file)            
            elif embedding_type=="text_embedding":
                    return client.get_text_embedding(text)
        except Exception as e:
            attempt += 1
            backoff_delay = min(2 ** attempt + random.uniform(0, 1), 32)  # Exponential backoff with jitter
            print(f"Attempt {attempt} failed with error {e}. Retrying in {backoff_delay:.2f} seconds...")
            time.sleep(backoff_delay)  # Wait before retrying

    raise Exception("Max retries reached. Could not complete the request.")

    
async def generate_query_embedding(client,text=None,image_file=None, embedding_type=None):
    try:
        # Retry logic with exponential backoff to calculate query embeddings
        result = exponential_backoff_retries(embedding_client, text, image_file, embedding_type)
        
        # Respond with the successful embedding response
        return {
            "text_embedding": result.text_embedding,
            "image_embedding": result.image_embedding
        }, 200

    except Exception as e:
        # Handle failure after max retries
        return f"Error: {str(e)}", 500


async def get_media_nearest_neighbors(query_embedding, table, dataset,source_embedding_column,project_id,top_k=50, filter_query=""):
    """Query nearest neighbors using cosine similarity in BigQuery for multimodal embeddings."""
    
    # Record the start time
    start_time = time.time()
    #options="""'{"fraction_lists_to_search": 0.5}'"""
    options="""'{"use_brute_force":true}' """

    sql = f"""  
         WITH search_results AS
         (
            SELECT
            search_results.base.uri as fileUri,  
            search_results.distance,  -- The computed distance (similarity score) between the embeddings
            search_results.base.asset_id ,
            search_results.base.ml_generate_embedding_start_sec as startOffset_seconds,
            search_results.base.ml_generate_embedding_end_sec as endOffset_seconds,  
            search_results.base.content_type as asset_type,
            search_results.base.image_license_type,
            search_results.base.publisher_type,
            search_results.base.photographer,
            search_results.base.date_published,
            search_results.base.dxcId,
            ROW_NUMBER() OVER (PARTITION BY search_results.base.asset_id ORDER BY distance ASC) AS rank_within_document  -- Rank by distance within each document
              
            FROM
              VECTOR_SEARCH(     
                ( SELECT 
                  uri,
                  asset_id,
                  ml_generate_embedding_start_sec,
                  ml_generate_embedding_end_sec,
                  content_type,
                  image_license_type,
                  publisher_type,
                  photographer,
                  date_published,
                  dxcId ,
                  ml_generate_embedding_result 

                  FROM  `{dataset}.{table}` WHERE 1=1 {filter_query}), --source embedding table
                '{source_embedding_column}',  -- Column with the embedding vectors in the base table

                -- Use the query embedding computed in the previous step
                 (SELECT {json.dumps(query_embedding)} query_embedding),  -- The query embedding from the CTE (query_embedding)

                -- Return top-k closest matches (adjust k as necessary)
                top_k =>{ top_k  }, -- Top k most similar matches based on distance
                distance_type => 'COSINE',
                options => {options}                
              ) search_results
              
          )
          SELECT 
            fileUri,  
            distance, 
            asset_id ,
            startOffset_seconds,
            endOffset_seconds,  
            asset_type,
            image_license_type,
            publisher_type,
            photographer,
            date_published,
            dxcId,
            rank_within_document  -- Rank by distance within each document
              
              
    """       
    #print(sql)
    bq_client = bigquery.Client(project_id)
  
    # Run the query
    query_job = bq_client.query(sql)

    # Fetch results
    results = query_job.result()
    
    output=[]
    for row in results:
        output.append({'asset_id':row['asset_id'], 'fileUri':row['fileUri'], "time_lines":row['time_lines'], "asset_type":row["asset_type"], "distance":row['min_distance'],
                       "headline":row["headline"],
                        "image_license_type":row['image_license_type'],
                        "publisher_type":row['publisher_type'],
                        "photographer":row['photographer'],
                        "date_published":str(row['date_published']),
                        "dxcId":row['dxcId']
                        })

    
    end_time = time.time()

    # Calculate the elapsed time
    elapsed_time = end_time - start_time
    print(elapsed_time)
    return output

async def get_content_nearest_neighbors(query_embedding, table, dataset,source_embedding_column,project_id,top_k=50,filter_query=""):
    """Query nearest neighbors using cosine similarity in BigQuery for text embeddings."""
    
    # Record the start time
    start_time = time.time()
    #options="""'{"fraction_lists_to_search": 0.5}'"""
    options="""'{"use_brute_force":true}' """
    
    sql = f"""  
         WITH search_results AS
         (
            SELECT
      
            distance,  -- The computed distance (similarity score) between the embeddings
            search_results.base.asset_id,
            search_results.base.headline,
            ifnull(search_results.base.html_safe_text,search_results.base.description) as description,
            search_results.base.startOffset_seconds,
            search_results.base.endOffset_seconds,
            search_results.base.fileUri,
            search_results.base.asset_type,    
            search_results.base.first_published_timestamp,
            search_results.base.brand_type,
            search_results.base.primary_category_name,
            search_results.base.byline[SAFE_OFFSET(0)].author_name,
            search_results.base.image_license_type,
            search_results.base.publisher_type,
            search_results.base.photographer,
            search_results.base.date_published,
            search_results.base.dxcId,
            ROW_NUMBER() OVER (PARTITION BY  search_results.base.asset_id ORDER BY distance ASC) AS rank_within_document  -- Rank by distance within each document
              
            FROM
              VECTOR_SEARCH(     
                ( SELECT 

                        asset_id,                       
                        headline,
                        html_safe_text,
                        description,
                        startOffset_seconds,
                        endOffset_seconds,
                        fileUri,
                        asset_type,
                        first_published_timestamp,
                        brand_type,
                        primary_category_name,
                        byline,
                        image_license_type,
                        publisher_type,
                        photographer,
                        date_published,
                        dxcId,
                        text_embedding_result                
                 FROM  `{dataset}.{table}` WHERE 1=1 {filter_query}), --source embedding table
                '{source_embedding_column}',  -- Column with the embedding vectors in the base table

                -- Use the query embedding computed in the previous step
                 (SELECT {json.dumps(query_embedding)} query_embedding),  -- The query embedding from the CTE (query_embedding)

                -- Return top-k closest matches (adjust k as necessary)
                top_k =>{ top_k  }, -- Top k most similar matches based on distance
                distance_type => 'COSINE',
                options => {options}                   
              ) search_results              
          )
          
          SELECT
          
            distance, 
            asset_id,
            CASE WHEN LOWER(asset_type) like '%video%' then
                 CONCAT(' ',
                   CASE  
                       WHEN UPPER(SPLIT(REPLACE(sr.asset_id,'SYD-NINE_','') ,'_')[OFFSET(0)])  LIKE '%NNNT%' THEN 'NINE NEWS 2023'  
                       WHEN UPPER(SPLIT(REPLACE(sr.asset_id,'SYD-NINE_','') ,'_')[OFFSET(0)]) LIKE '%MAAT2023%' THEN 'MARRIED AT FIRST SIGHT 2023'
                       WHEN UPPER(SPLIT(REPLACE(sr.asset_id,'SYD-NINE_','') ,'_')[OFFSET(0)]) LIKE '%60MI23%' THEN '60 MINUTES 2023'
                   END ,
                   ' EPISODE ' , UPPER(SPLIT(REPLACE(sr.asset_id,'SYD-NINE_','') ,'_')[OFFSET(1)])
                   )
            ELSE IFNULL(sr.headline,'') END AS headline, 
            description,
            startOffset_seconds,
            endOffset_seconds,
            fileUri,
            asset_type,    
            first_published_timestamp,
            brand_type,
            primary_category_name,
            author_name,
            image_license_type,
            publisher_type,
            photographer,
            date_published,
            dxcId 
            
        FROM search_results
    """       
 
    #print(sql)
    bq_client = bigquery.Client(project_id)
  
    # Run the query
    query_job = bq_client.query(sql)

    # Fetch results
    results = query_job.result()  
    df = results.to_dataframe()
    
    # calculate min distance per asset
    ranked_documents = df.groupby('asset_id')['distance'].min().reset_index()
    ranked_documents = ranked_documents.rename(columns={'distance': 'min_distance'})

    print(ranked_documents)

    output=[]
    for row in results:     
        output.append({'asset_id':row['asset_id'], 'headline':row['headline'],'description':row['description'],'fileUri':row['fileUri'], "time_lines":row['time_lines'], "asset_type":row["asset_type"], 
                      "distance":row['min_distance'],
                       "first_published_timestamp":row['first_published_timestamp'].isoformat() if not row['first_published_timestamp'] is None else row['first_published_timestamp']  ,
                        "brand_type":row['brand_type'],
                        "primary_category_name":row['primary_category_name'],
                        "author_name":row['author_name'],
                        "image_license_type":row['image_license_type'],
                        "publisher_type":row['publisher_type'],
                        "photographer":row['photographer'],
                        "date_published":str(row['date_published']),
                        "dxcId":row['dxcId']
                      
                      })
  
    end_time = time.time()
    # Calculate the elapsed time
    elapsed_time = end_time - start_time
    print(elapsed_time)
    return output

def merge_result(combined_list):
    # Step 2: Create a dictionary to merge by 'id'
    merged_dict = {}

    # Step 3: Iterate through the combined list and merge dictionaries by 'id'
    for d in combined_list:
        id_value = d['asset_id']

        # If the id already exists in merged_dict, update it
        if id_value in merged_dict:
            merged_dict[id_value].update(d)
        else:
            # If the id doesn't exist, add the dictionary as it is
            merged_dict[id_value] = d.copy()

    # Step 4: Convert the merged dictionary back into a list
    final_merged_list = list(merged_dict.values())
    
    return final_merged_list



async def get_nearest_contet(request):
    """
    Cloud Function entry point. This function handles the incoming request, 
    performs exponential backoff retries, and returns the embedding response.
    """ 
    # Parse the incoming request to extract text or image file
    request_json = request.get_json(silent=True)
    if "search_query" in request_json:
        text = request_json.get('search_query')
    else:
        text=""

    if "image_file"  in request_json:
        image_file = request_json.get('image_file')  # Assume it's the path or base64 string of the image
    else:
        image_file=""

   

    project_id = request_json.get('project')  
    region = request_json.get('region')  

    if "filter_image" in request_json: 
       filter_image = request_json.get('filter_image') 
    else:
        filter_image="True"

    if "filter_video" in request_json:
        filter_video = request_json.get('filter_video') 
    else:
        filter_video="True"
    
    if "filter_article" in request_json:
       filter_article=request_json.get('filter_article')
    else:
        filter_article="True"
    
    # Load configuration from config.json
    with open('config.json') as config_file:
         config = json.load(config_file)
    
    
    top_k=int(config['top_k'])  
    dataset= config['dataset']
    content_table=config['content_table']
    mm_table=config['mm_table']
    content_source_embedding_column=config['content_source_embedding_column']
    mm_source_embedding_column=config['mm_source_embedding_column'] 
    if image_file=="" or image_file=="None" or image_file==None:
        image_file=None
    else:
        image_file = base64.b64decode(image_file)
            
       

    # Initialize the EmbeddingPredictionClient outside the function for reuse
    embedding_client = EmbeddingPredictionClient(project=project_id , location=region,api_regional_endpoint=region+"-aiplatform.googleapis.com")
        
    if (text=="") and (image_file==None):
        return 'Error: At least one of "text" or "image_file" must be provided.', 400
     
    content_result_article=[]
    content_result_image=[]
    content_result_video=[]
    media_image_result=[]
    media_video_result=[]
    error=""
    try:
        if text:
            article_filter_query=""
            if filter_article=="True" or filter_article=="1":
                article_filter_query= article_filter_query+f" AND lower(asset_type) like '%article%' " 
                
            image_filter_query=""
            if filter_image=="True" or filter_image=="1":
                image_filter_query= image_filter_query+f" AND lower(asset_type) like '%image%' "  
                
            video_filter_query=""
            if filter_video=="True" or filter_video=="1":
                video_filter_query= video_filter_query+f" AND lower(asset_type) like '%video%' "

            #if a text is given, calculate both multiomdal embedding and text embedding of the search query
            txtembding_for_text_result =  await asyncio.create_task(exponential_backoff_retries(embedding_client, text, embedding_type='text_embedding'))
            #mmembding_for_text_result =  await asyncio.create_task(exponential_backoff_retries(embedding_client, text, embedding_type='multimodal_embedding')) 
            txtembding_for_text_result=txtembding_for_text_result .text_embedding
            #mmembding_for_text_result=mmembding_for_text_result.text_embedding
            #find nearest neighbours both from text embedding and multimodal embedding
            if article_filter_query!="":
                content_result_article = asyncio.create_task(get_content_nearest_neighbors(txtembding_for_text_result, content_table, dataset,content_source_embedding_column,project_id,top_k=top_k, filter_query=article_filter_query))
            if image_filter_query!="":
                content_result_image = asyncio.create_task(get_content_nearest_neighbors(txtembding_for_text_result, content_table, dataset,content_source_embedding_column,project_id,top_k=top_k,filter_query=image_filter_query))
            if video_filter_query!="":
                content_result_video = asyncio.create_task(get_content_nearest_neighbors(txtembding_for_text_result, content_table, dataset,content_source_embedding_column,project_id,top_k=top_k, filter_query=video_filter_query))
            
            if article_filter_query!="":
                    content_result_article= await(content_result_article)
            if image_filter_query!="":
                    content_result_image=await(content_result_image)
            if video_filter_query!="":
                     content_result_video=await(content_result_video)
       

        if image_file: 

            image_filter_query=""
            if filter_image=="True" or filter_image=="1":
                image_filter_query= image_filter_query+f" AND lower(content_type) like '%image%' "  
                
            video_filter_query=""
            if filter_video=="True" or filter_video=="1":
                video_filter_query= video_filter_query+f" AND lower(content_type) like '%video%' "

            #if an image is given convert image to 64bytestring and extract embedding
            mmembding_for_image_result = await asyncio.create_task(exponential_backoff_retries(embedding_client, text="",image_file=image_file, embedding_type='multimodal_embedding'))
            mmembding_for_image_result=mmembding_for_image_result.image_embedding
            #find nearest neighbours both from multimodal embedding

            if image_filter_query!="":
                media_image_result = asyncio.create_task(get_media_nearest_neighbors(mmembding_for_image_result, mm_table, dataset,mm_source_embedding_column,project_id,top_k=top_k,filter_query=image_filter_query))
            if video_filter_query!="":
                media_video_result = asyncio.create_task(get_media_nearest_neighbors(mmembding_for_image_result, mm_table, dataset,mm_source_embedding_column,project_id,top_k=top_k,filter_query=video_filter_query))
            
            if image_filter_query!="":
                media_image_result = await media_image_result
            if video_filter_query!="":
                media_video_result = await media_video_result

    except Exception as e:
        error= str(e)

    

    final_merged_list=merge_result(content_result_article+content_result_image+content_result_video+media_image_result+media_video_result)
    #log_data(final_merged_list,error,request_json ,0,project_id)
    return final_merged_list,error, project_id


def log_data(result,error,request,elapsed_time,project_id):
    """
      Log the search result into bigquery
    Args:
       List[dict]  result: the result of search
       str error: the error message
       dict request: the request sent
       float elapsed_time: the time taken for the search result to be generated
       str project_id: project id
    """
    rows_to_insert=[] 
    rows_to_insert.append(
                                {  "search_date":  datetime.now().isoformat() ,
                                    "request":request,
                                    "response":   result  , 
                                    "error":  error,
                                    "elapsed_time":elapsed_time ,
                                    "API": "Search"
                                  
                                    }
                                            )   

    #create table new if does not exist
    # Load configuration from config.json
    with open('config.json') as config_file:
         config = json.load(config_file)

    table=config['log_table']
    dataset_id=config['log_dataset']
    #push the data into the table
    table_id = f"{project_id}.{dataset_id}.{table}"
    client = bigquery.Client(project_id)
    dataset  = client.dataset(dataset_id)
    table = dataset.table(table)
    # job_config = bigquery.LoadJobConfig()
    # job_config.source_format = bigquery.SourceFormat.NEWLINE_DELIMITED_JSON
    # job_config.schema = table_schema
    job = client.load_table_from_json(rows_to_insert, table) 

           
    

# @functions_framework.http
# async def search_content_function(request):
 
#     result = await get_nearest_contet(request) 
#     return result#[0],result[1],result[2]

@functions_framework.http
def search_content_function(request):
    """This is the entry point for the Cloud Function."""
    try:
        loop = asyncio.get_event_loop()
    except RuntimeError as e:
        # If no event loop is running, create a new event loop for this thread
        loop = asyncio.new_event_loop()
        asyncio.set_event_loop(loop)
    
    start_time = time.time()
    result,error,project_id = loop.run_until_complete(get_nearest_contet(request))
    end_time = time.time()
    # Calculate the elapsed time
    elapsed_time = end_time - start_time

    #record the search log
    log_data(result,error,request.get_json(silent=True),elapsed_time,project_id)
    return result
     

In [231]:
x =await search_content_function('')

4.155578851699829
1.2137744426727295
search is done


In [203]:
response_textembedding=exponential_backoff_retries(embedding_client, 'Curtis Sittenfeld', embedding_type='text_embedding').text_embedding


In [76]:
response_multimodal_embedding=exponential_backoff_retries(embedding_client, 'Curtis sittenfeld', embedding_type='multimodal_embedding').text_embedding

In [91]:
###text embedding
import asyncio


In [83]:
PROJECT = !gcloud config get-value project
PROJECT_ID = PROJECT[0]
REGION = "us-central1" 

In [84]:
query_embedding=response_textembedding
top_k=50
project_id=PROJECT_ID
dataset='langchain_dataset'
table='vlt_media_content_text_test_for_search'
source_embedding_column='ml_generate_embedding_result'

In [211]:
import asyncio
 
if 1==1:
    
    query_embedding=response_textembedding
    top_k=50
    project_id=PROJECT_ID
    dataset='langchain_dataset'
    table='vlt_media_content_text_test_for_search'
    source_embedding_column='ml_generate_embedding_result'

    
    #content_result= 
    
    # Create tasks and run them concurrently
    content_result = await asyncio.create_task(get_content_nearest_neighbors(query_embedding, table, dataset,source_embedding_column,project_id,top_k=50))
    
    query_embedding=response_multimodal_embedding
    top_k=50
    project_id=PROJECT_ID
    dataset='vlt_media_embeddings_integration'
    table='vlt_imgvdo_multimodal_embeddings'
    source_embedding_column='ml_generate_embedding_result'
    media_result = await asyncio.create_task(get_media_nearest_neighbors(query_embedding, table, dataset,source_embedding_column,project_id,top_k=50))


4.390771865844727
0.48775458335876465


In [13]:
sql="""WITH search_results AS
         (
            SELECT
            search_results.base.content as content,  
            distance,  -- The computed distance (similarity score) between the embeddings
            search_results.base.asset_id,
            search_results.base.headline,
            ifnull(search_results.base.html_safe_text,search_results.base.description) as description,
            search_results.base.startOffset_seconds,
            search_results.base.endOffset_seconds,
            search_results.base.fileUri,
            search_results.base.asset_type,    
            search_results.base.first_published_timestamp,
            search_results.base.brand_type,
            search_results.base.primary_category_name,
            search_results.base.byline[SAFE_OFFSET(0)].author_name,
            search_results.base.image_license_type,
            search_results.base.publisher_type,
            search_results.base.photographer,
            search_results.base.date_published,
            search_results.base.dxcId,
            ROW_NUMBER() OVER (PARTITION BY  search_results.base.asset_id ORDER BY distance ASC) AS rank_within_document  -- Rank by distance within each document
              
            FROM
              VECTOR_SEARCH(     
                ( SELECT 

                        asset_id, 
                        content,
                        headline,
                        html_safe_text,
                        description,
                        startOffset_seconds,
                        endOffset_seconds,
                        fileUri,
                        asset_type,
                        first_published_timestamp,
                        brand_type,
                        primary_category_name,
                        byline,
                        image_license_type,
                        publisher_type,
                        photographer,
                        date_published,
                        dxcId,
                        text_embedding_result                
                 FROM  `vlt_media_embeddings_integration.vlt_all_media_content_text_embeddings` WHERE 1=1  AND lower(asset_type) like '%video%' ), --source embedding table
                'text_embedding_result',  -- Column with the embedding vectors in the base table

                -- Use the query embedding computed in the previous step
                 (SELECT [0.005493919365108013, -0.07423227280378342, -0.056699592620134354, -0.05905818194150925, 0.052528224885463715, -0.029050879180431366, -0.03154755383729935, -0.0014369161799550056, 0.05284561216831207, 0.011417239904403687, 0.00924063939601183, -0.0054649473167955875, 0.00799805112183094, -0.02406947687268257, 0.026166586205363274, 0.0029337643645703793, 0.044714219868183136, 0.05616328865289688, 0.0022112263832241297, -0.016903135925531387, 0.019320717081427574, -0.012929008342325687, 0.006841930560767651, -0.0438101589679718, -0.011321177706122398, -0.030933387577533722, 0.06392339617013931, -0.1103433221578598, -0.046109799295663834, 0.029660629108548164, -0.05990747734904289, 0.029502801597118378, -0.03766076639294624, 0.041455287486314774, 0.005668437574058771, -0.061352234333753586, 0.030761457979679108, -0.04995541647076607, 0.03407169133424759, 0.043515194207429886, -0.016968755051493645, 0.0020632522646337748, -0.021038778126239777, -0.004070797003805637, -0.025316109880805016, 0.03915589675307274, 0.010867534205317497, -0.018197866156697273, 0.005580325145274401, -0.03899497911334038, 0.021638676524162292, -0.011973653919994831, 0.08048750460147858, -0.050325553864240646, -0.007891271263360977, 0.014494232833385468, 0.08448202162981033, -0.014370097778737545, -0.03325944393873215, -0.035708948969841, 0.05086464062333107, 0.02465665526688099, -0.04220596328377724, -0.025965269654989243, -0.0023405365645885468, -0.0008994092931970954, -0.02000521309673786, 0.003716134000569582, 0.09658436477184296, 0.006453088019043207, 0.03432152792811394, -0.02381335385143757, 0.060877345502376556, -0.009956647641956806, 0.007767858449369669, -0.11898826062679291, -0.05264763534069061, 0.003996422979980707, -0.01120732631534338, 0.05962861701846123, 0.025364473462104797, -0.04393245652318001, -0.024399681016802788, 0.004163143690675497, -0.08431538194417953, -0.0027924268506467342, 0.009860184043645859, 0.03654220327734947, 0.023205427452921867, 0.04184042662382126, -0.04729235917329788, 0.03979603946208954, 0.0068242717534303665, -0.07453237473964691, 0.015338048338890076, 0.06438129395246506, -0.06548930704593658, -0.01093421969562769, 0.039066754281520844, 0.0038349556270986795, 0.01714404672384262, -0.04105100408196449, 0.037275586277246475, 0.03967505320906639, 0.044278983026742935, 0.021809149533510208, 0.014105798676609993, 0.03656208887696266, 0.034777265042066574, 0.014304626733064651, -0.1014489009976387, 0.011369454674422741, -0.025156408548355103, 0.002635310171172023, 0.04157416895031929, -0.0004785340279340744, -0.03367149084806442, 0.0884716585278511, 0.011085143312811852, 0.018607504665851593, 0.04754510149359703, -0.004499243106693029, 0.054413795471191406, 0.013694092631340027, -0.0009129418176598847, 0.004069854971021414, 0.00881060678511858, 0.023358464241027832, -0.020025214180350304, 0.03049168922007084, -0.05306631699204445, -0.032905708998441696, 0.014962037093937397, -0.0044717565178871155, 0.05538986623287201, 0.06424450129270554, 0.056417398154735565, 0.005774857010692358, 0.047539617866277695, -0.003262350568547845, 0.01686561107635498, 0.027184484526515007, -0.002625703811645508, -0.026795340701937675, 0.04740332439541817, 0.05231247469782829, 0.012403270229697227, -0.06964562833309174, 0.03340134024620056, -0.0069298758171498775, -0.015346331521868706, -0.03144174441695213, -0.022034071385860443, -0.03513352572917938, 0.04940056428313255, 0.003328869352117181, -0.031684499233961105, -0.026252202689647675, -0.027722079306840897, 0.02771230787038803, 0.01594596914947033, -0.009434619918465614, -0.01709051802754402, 0.00781565997749567, 0.020092401653528214, 0.022676222026348114, -0.01729520410299301, 0.016962755471467972, -0.010072060860693455, 0.0025325806345790625, -0.0014186697080731392, 0.019845396280288696, -0.028982918709516525, -0.059151485562324524, 0.005259308498352766, -0.06030282750725746, 0.01997550018131733, 0.04506336897611618, -0.032182544469833374, -0.00757491122931242, -0.04923570156097412, -0.032132040709257126, -0.009899057447910309, 0.0029140706174075603, 0.012371824122965336, 0.001693424885161221, 0.019387707114219666, -0.059771135449409485, -0.005411704070866108, 0.03327430412173271, -0.009559409692883492, -0.0023384327068924904, -0.06554350256919861, 0.009303078055381775, 0.009706953540444374, 0.052617885172367096, -0.008516823872923851, 0.02569408155977726, -0.00024115925771184266, 0.00010570079030003399, -0.03661264479160309, 0.14422643184661865, 0.003794020740315318, 0.02769950032234192, 0.015411059372127056, 0.025315478444099426, 0.03990370035171509, 0.00750946206972003, -0.018215743824839592, -0.00012583537318278104, -0.06935805082321167, 0.0018884779419749975, -0.00869299378246069, -0.0027066171169281006, 0.046640411019325256, 0.03275663033127785, -0.032275740057229996, -0.0009171852143481374, -0.026764411479234695, 0.002334192395210266, -0.0228977520018816, 0.023303931578993797, -0.03144214674830437, 0.01593603752553463, -0.026322871446609497, 0.01856415346264839, -0.05038043111562729, 0.04166223853826523, 0.038867734372615814, -0.017571287229657173, 0.03859161585569382, 0.036519855260849, 0.048887912184000015, 0.017132190987467766, 0.060866232961416245, 0.004635718185454607, -0.001499234582297504, -0.02169746346771717, 0.014184855856001377, 0.06766337156295776, -0.008984933607280254, 0.021984664723277092, 0.015995172783732414, -0.011528050526976585, -0.009745053946971893, -0.05121775344014168, 0.003736746497452259, 0.05131399258971214, 0.004999445751309395, -0.041059285402297974, 0.025519441813230515, -0.026338769122958183, 0.0006980240577831864, 0.04730502888560295, -0.08033531904220581, 0.0347997285425663, -0.07263945043087006, -0.0003268693690188229, -0.02429404854774475, -0.008424067869782448, 0.031000178307294846, -0.018200131133198738, -0.03607449680566788, -0.0341821052134037, -0.018239809200167656, 0.024189498275518417, 0.029842520132660866, -0.04998728632926941, 0.029021458700299263, 0.011902113445103168, -0.024955617263913155, 0.004741176031529903, 0.0647798627614975, 0.011564671993255615, -0.025873804464936256, 0.0017046962166205049, 0.04701589420437813, 0.0018561403267085552, -0.051033370196819305, -0.0020389268174767494, 0.044943343847990036, 0.0749484971165657, 0.04729871451854706, -0.06013672426342964, 0.009848801419138908, 0.009316415525972843, -0.03293829783797264, 0.03683047741651535, 0.016398660838603973, -0.02647230215370655, -0.043802473694086075, -0.023871613666415215, -0.009661915712058544, 0.016663335263729095, -0.03971428424119949, 0.001310832449235022, -0.03843007981777191, 0.026431189849972725, -0.0030461696442216635, -0.052180252969264984, -0.0068421512842178345, -0.11140625178813934, -0.006113843526691198, -0.0227365605533123, -0.017762934789061546, 0.04095969721674919, -0.012908424250781536, -0.06462644785642624, 0.029798636212944984, -0.0005301124765537679, -0.0359497033059597, 0.00732902018353343, -0.05055161193013191, -0.01591498963534832, 0.04838280752301216, 0.02384313941001892, -0.00588661665096879, 0.02319006808102131, 0.0056687393225729465, -0.030990781262516975, -0.000682059268001467, 0.038905784487724304, -0.0028312329668551683, -0.00410960428416729, -0.025027787312865257, -0.0005802505766041577, -0.07170005142688751, 0.013733630999922752, 0.03357236459851265, -0.008499776013195515, -0.023045480251312256, -0.04984239116311073, -0.02539234049618244, 0.025440191850066185, 0.00616514403373003, 0.02552996575832367, -0.0507466122508049, -0.010218429379165173, -0.018101299181580544, 0.04987122118473053, 0.04977365955710411, 0.007653119973838329, -0.015694037079811096, -0.0746794044971466, -0.01754187047481537, -0.004645380191504955, -0.0025611252058297396, 0.015548431314527988, 0.05624477565288544, 0.027049794793128967, 0.02129197306931019, 0.08540055900812149, -0.03295433148741722, -0.024050559848546982, 0.03560066595673561, -0.04090002179145813, -0.01174505241215229, -0.04305324703454971, 0.013771958649158478, -0.01866539567708969, -0.0316360667347908, 0.023620454594492912, 0.008586048148572445, -0.03565482050180435, -0.024475064128637314, -0.023056816309690475, 0.010912375524640083, -0.02171836607158184, 0.001386574236676097, -9.071022941498086e-05, -0.009197898209095001, -0.03904204070568085, 0.07763411849737167, -0.007219633553177118, -0.05769161507487297, -0.013007770292460918, -0.03703181445598602, -0.022313253954052925, 0.013910877518355846, 0.017865469679236412, 0.017986541613936424, -0.02979075349867344, 0.025980180129408836, 0.03260488063097, 0.055688098073005676, 0.00750646973028779, -0.00515609560534358, 0.020837340503931046, 0.0472932793200016, -0.011295901611447334, 0.00961307156831026, 0.022110335528850555, 0.04528600350022316, -0.014450023882091045, 0.03346050903201103, -0.0003139156033284962, 0.018720723688602448, -0.03317435458302498, -0.007475060876458883, -0.03289001062512398, -0.00893274787813425, 0.002032813848927617, -0.06530139595270157, -0.02767796441912651, 0.010828891769051552, -0.009120892733335495, 0.016258085146546364, -0.048568811267614365, 0.016024906188249588, 0.002256694482639432, 0.05232146009802818, 0.039309944957494736, 0.012412735261023045, -0.05459079146385193, -0.08584057539701462, -0.01186318602412939, 0.062495384365320206, -0.04176274314522743, 0.007162642665207386, 0.03613175079226494, -0.037958040833473206, 0.0016471444396302104, -0.024343200027942657, 0.02695823274552822, -0.011697963811457157, -0.06637656688690186, 0.024505818262696266, -0.015689363703131676, 0.046842675656080246, 0.011939444579184055, 0.027445973828434944, 0.030265625566244125, -0.005165617447346449, -0.02450387552380562, -0.021929357200860977, -0.0002695195435080677, 0.010983840562403202, 0.03937280923128128, -0.007310978136956692, 0.006166358012706041, 0.013443493284285069, -0.03074072301387787, 0.02771095372736454, -0.018069809302687645, -0.017553415149450302, -0.04904952272772789, -0.0071190120652318, -0.026859167963266373, 0.05115056037902832, -0.07120644301176071, 0.04361604526638985, -0.07017751783132553, -0.0036651052068918943, -0.05740373209118843, -0.004306306596845388, -0.024133848026394844, 0.03296871855854988, 0.08780230581760406, -0.048599183559417725, -0.034816499799489975, -0.03164049610495567, -0.013695602305233479, -0.09225906431674957, -0.11194275319576263, 0.01888064295053482, -0.010553937405347824, 0.0472041554749012, 0.009713256731629372, 0.002513392362743616, 0.01549038290977478, 0.010732552967965603, -0.04318328946828842, -0.033553287386894226, -0.018366876989603043, -0.015557334758341312, -0.015357859432697296, -0.04336283728480339, 0.00861707516014576, 0.019759399816393852, -0.023216594010591507, 0.005117423832416534, -0.0310193058103323, 0.09827026724815369, 0.05559295043349266, -0.024114267900586128, -0.007579607889056206, 0.0028337216936051846, -0.07734467834234238, -0.048742447048425674, 0.022629505023360252, -0.0055853622034192085, -0.044031817466020584, -0.026611676439642906, 0.007583355065435171, -0.007459694519639015, 0.002837469568476081, 0.006242482922971249, 0.054717473685741425, -0.007605109363794327, 0.020721333101391792, 0.003833017311990261, 0.014132507145404816, 0.05379598215222359, 0.004835441242903471, 0.05135606974363327, -0.0837629958987236, -0.009585377760231495, 0.027926338836550713, -0.04175431281328201, 0.013916486874222755, -0.02245919592678547, -0.0018881496507674456, -0.003046636702492833, -0.0005293331341817975, 0.04004726558923721, -0.015025629661977291, -0.0036162640899419785, 0.015825243666768074, 0.0687679722905159, -0.03017687052488327, 0.05273747816681862, -0.004234414082020521, -0.06907638162374496, 0.042406998574733734, 0.0007894746377132833, -0.045363329350948334, -0.004390369169414043, 0.02519507147371769, -0.06327027827501297, -0.011371063999831676, -0.025887979194521904, 0.04976749047636986, -0.05300825089216232, 0.01108279824256897, 0.015833614394068718, -0.04075346514582634, -0.010968007147312164, 0.04531209170818329, 0.0020068257581442595, -0.0028414824046194553, 0.03684569522738457, -0.03711998090147972, 0.024422870948910713, 0.024984875693917274, -0.017047693952918053, -0.005211206153035164, 0.07278819382190704, -0.07777056843042374, 0.002119366778060794, 0.003415257204324007, -0.02951631136238575, -0.019113529473543167, 0.05613609775900841, -0.026049144566059113, 0.02858898974955082, 0.031009221449494362, -0.016346175223588943, 0.010753247886896133, 0.04291897267103195, 0.0031291008926928043, -0.03123539686203003, -0.003060722490772605, 0.004954046569764614, -0.010697318241000175, 0.046114057302474976, 0.022749165073037148, -0.009316925890743732, -0.005225364584475756, 0.033718399703502655, -0.014649983495473862, -0.053425759077072144, -0.025177404284477234, 0.08267928659915924, -0.025288065895438194, 0.06340519338846207, 0.015423402190208435, -0.023190226405858994, 0.0022070021368563175, 0.011445406824350357, 0.002066484885290265, 0.023386042565107346, -0.06387845426797867, 0.0005388004938140512, 0.049076952040195465, -0.014603378251194954, 0.0069856103509664536, 0.024084486067295074, -0.0013131239684298635, 0.044620055705308914, -0.03761081025004387, -0.05296264961361885, -0.016176719218492508, -0.035641152411699295, 0.003475508652627468, -0.027263114228844643, -0.012199478223919868, -0.018861321732401848, -0.0632835403084755, -0.012765246443450451, -0.006964516825973988, 0.06140752136707306, -0.08310317248106003, 0.07117675244808197, -0.000736712128855288, 0.0042215147987008095, -0.011258806101977825, 0.02437269501388073, -0.052948836237192154, 0.006284818518906832, 0.025754446163773537, -0.009142535738646984, 9.004119056044146e-05, 0.01484593003988266, 0.009541717357933521, -0.08811461180448532, 0.003316933987662196, 0.02411179058253765, 0.025048067793250084, 0.02226690575480461, -0.04031919315457344, -0.01508781686425209, 0.02252103202044964, -0.010594488121569157, -0.012138580903410912, 0.046527199447155, 0.012063958682119846, -0.01040472649037838, 0.011079162359237671, 0.05778948590159416, 0.026396164670586586, 0.07364512234926224, 0.03233606740832329, 0.0030453666113317013, 0.027655377984046936, -0.017093058675527573, 0.03829348832368851, 0.006334786303341389, 0.019041690975427628, -0.017924200743436813, -0.022899098694324493, -0.060510698705911636, -0.004535946995019913, 0.06738438457250595, -0.008143501356244087, -0.015487953089177608, 0.08035858720541, 0.02043013460934162, -0.05807823687791824, -0.06448059529066086, -0.003932093270123005, -0.026611948385834694, 0.021811099722981453, 0.02509414590895176, 0.027344992384314537, -0.0003643052186816931, -0.004700418096035719, -0.006619364954531193, 0.008219418115913868, 0.02398563362658024, -0.018301138654351234, -0.07367381453514099, -0.028014829382300377, -0.02473388984799385, -0.04725584387779236, 0.010061023756861687, -0.03113120049238205, -0.07590296864509583, -0.08779522031545639, 0.034827958792448044, -0.01374551746994257, -0.07895327359437943, 0.027294373139739037, 0.04027180001139641, -0.027629584074020386, 0.05207102745771408, 0.02324972115457058, -0.015338659286499023, 0.04337887093424797, -0.04513213038444519, 0.02012067846953869, 0.007027043960988522, -0.061791423708200455, 0.007051311898976564, -0.022894009947776794, 0.002565824892371893, 0.007414116524159908, 0.004596408922225237, -0.035960614681243896, -0.01810777746140957, -0.032050881534814835, 0.026434164494276047, -0.00952052790671587, -0.05146074667572975, 0.0029895054176449776, 0.03544249385595322, 0.02052290365099907, -0.05748838186264038, 0.002209521597251296, -0.023602081462740898, 0.05138986557722092, -0.012100444175302982, -0.017674988135695457, -0.011695045977830887, 0.02864970825612545, -0.037440478801727295, -0.012585175223648548, -0.011415270157158375, 0.05010531097650528, -0.0074990857392549515, 0.04714967682957649, 0.014191143214702606, -0.04783186316490173, -0.04829198494553566, 0.03674681484699249, 0.08626721799373627, -0.01016426645219326, -0.005536897573620081, 0.01655212789773941, -0.009703002870082855, 0.043782610446214676, 0.02154373936355114, 0.02895435318350792, 0.047658443450927734, -0.014723381027579308, -0.07162956148386002, 0.07207208126783371, -0.058180391788482666, 0.03169959411025047, -0.011348799802362919, 0.0011657638242468238, 0.018125493079423904, -0.04168395698070526, -0.009067137725651264, 0.027917595580220222, -0.02737058512866497, 0.04669126123189926, 0.02220216393470764, 0.05528002232313156, 0.012885313481092453, -0.05991406366229057, -0.02789248712360859, 0.022814977914094925, 0.033465076237916946, 0.05884036049246788, 0.04963691160082817, -0.03900032863020897, 0.016441041603684425, -0.03141330927610397, -0.0023916326463222504, 0.003846655134111643, -0.0401528999209404, 0.0014873954933136702, 0.016952836886048317, 0.06659151613712311, 0.022433390840888023, -0.014014817774295807, 0.047221146523952484, -0.031017163768410683, -0.014675581827759743, 0.02566838078200817, -0.05482173338532448, 0.003715591738000512, 0.03588821366429329, 0.01702059432864189, 0.02009516954421997, 0.01457085832953453, -0.047865066677331924, 0.011643655598163605] query_embedding),  -- The query embedding from the CTE (query_embedding)

                -- Return top-k closest matches (adjust k as necessary)
                top_k =>50, -- Top k most similar matches based on distance
                distance_type => 'COSINE',
                options => '{"fraction_lists_to_search": 0.5}'                   
              ) search_results              
          )SELECT          
            distance, 
            asset_id,
            CASE WHEN LOWER(asset_type) like '%video%' then
                 CONCAT(' ',
                   CASE  
                       WHEN UPPER(SPLIT(REPLACE(asset_id,'SYD-NINE_','') ,'_')[OFFSET(0)])  LIKE '%NNNT%' THEN 'NINE NEWS 2023'  
                       WHEN UPPER(SPLIT(REPLACE(asset_id,'SYD-NINE_','') ,'_')[OFFSET(0)]) LIKE '%MAAT2023%' THEN 'MARRIED AT FIRST SIGHT 2023'
                       WHEN UPPER(SPLIT(REPLACE(asset_id,'SYD-NINE_','') ,'_')[OFFSET(0)]) LIKE '%60MI23%' THEN '60 MINUTES 2023'
                   END ,
                   ' EPISODE ' , UPPER(SPLIT(REPLACE(asset_id,'SYD-NINE_','') ,'_')[OFFSET(1)])
                   )
            ELSE IFNULL(headline,'') END AS headline, 
            description,
            startOffset_seconds,
            endOffset_seconds,
            fileUri,
            asset_type,    
            first_published_timestamp,
            brand_type,
            primary_category_name,
            author_name,
            image_license_type,
            publisher_type,
            photographer,
            date_published,
            dxcId 
            
        FROM search_results sr"""

In [117]:
if 1==1:
    #print(sql)
    bq_client = bigquery.Client('nine-quality-test')
  
    # Run the query
    query_job = bq_client.query(sql)

    # Fetch results
    results = query_job.result()  
    df = results.to_dataframe()
 
    # calculate min distance per asset
    min_distance_per_asset = df.groupby('asset_id')['distance'].min().reset_index()
    min_distance_per_asset = min_distance_per_asset.rename(columns={'distance': 'min_distance'})

    # Merge the min_distance_per_asset into the original DataFrame
    df = df.merge(min_distance_per_asset, on='asset_id', how='left')

    #calculate the min distance per segment
    min_distance_per_segment = df.groupby(['asset_id', 'startOffset_seconds', 'endOffset_seconds'])['distance'].min().reset_index()
    min_distance_per_segment = min_distance_per_segment.rename(columns={'distance': 'segment_min_distance'})
 
    # Merge the min_distance_per_offset into the original DataFrame
    df = df.merge(min_distance_per_segment, on=['asset_id','startOffset_seconds', 'endOffset_seconds'], how='left')
   
    #drop duplicates
    df = df.drop_duplicates(subset=['asset_id', 'headline', 'description',
           'startOffset_seconds', 'endOffset_seconds', 'fileUri', 'asset_type',
           'first_published_timestamp', 'brand_type', 'primary_category_name',
           'author_name', 'image_license_type', 'publisher_type', 'photographer',
           'date_published', 'dxcId', 'min_distance', 'segment_min_distance'])
    
    # Sort by asset_id and startOffset_seconds to ensure proper order
    df = df.sort_values(by=['asset_id', 'startOffset_seconds'])

    # Aggregate descriptions for each asset_id, ordered by startOffset_seconds
    df['description'] = df.groupby('asset_id')['description'].transform(lambda x: '\n'.join(x))

    # Aggregate and concatenate segments for each asset_id
    df['time_lines'] = df.apply(
        lambda row: f"{{'startOffset_seconds': {row['startOffset_seconds']}, 'endOffset_seconds': {row['endOffset_seconds']}, 'segment_min_distance': {row['segment_min_distance']}}}", axis=1)
        
    # Now group by 'asset_id' and concatenate the strings in 'time_lines'
    time_lines = df.groupby(['asset_id'])['time_lines'].apply(lambda x: ', '.join(x)).reset_index()
    
    df.drop('time_lines', axis=1, inplace=True)
    # Merge the time_lines into the original DataFrame
    df = df.merge(time_lines, on=['asset_id'], how='left')
   
    #drop duplicates
    df = df.drop_duplicates(subset=['asset_id', 'headline', 'description',
            'fileUri', 'asset_type',
           'first_published_timestamp', 'brand_type', 'primary_category_name',
           'author_name', 'image_license_type', 'publisher_type', 'photographer',
           'date_published', 'dxcId', 'min_distance','time_lines'])[['asset_id', 'headline', 'description',
            'fileUri', 'asset_type',
           'first_published_timestamp', 'brand_type', 'primary_category_name',
           'author_name', 'image_license_type', 'publisher_type', 'photographer',
           'date_published', 'dxcId', 'min_distance','time_lines']]
    # Convert datetime to string using astype(str)
    df['date_published'] = df['date_published'].astype(str)
    df['first_published_timestamp'] = df['first_published_timestamp'].astype(str)

    out_put = df.to_dict(orient='records')

/opt/conda/lib/python3.10/site-packages/google/cloud/bigquery/table.py:2309: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)
/opt/conda/lib/python3.10/site-packages/google/cloud/bigquery/table.py:2323: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)
/opt/conda/lib/python3.10/site-packages/google/cloud/bigquery/table.py:2337: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)


50


In [118]:
out_put

[{'asset_id': 'vlt_video_extract_MAAT_Full_MAAT2023_10_A_HBB.mp4',
  'headline': None,
  'description': "The video starts with Claire, a 31-year-old kindergarten assistant from Victoria, recalling a moment of intimacy with Jesse. She describes how their relationship progressed from initial awkwardness to a comfortable and natural connection. She reminisces about their wedding day, recalling the spark she felt when she first saw Jesse at the altar. The video then shows flashbacks of their wedding ceremony, highlighting their joy and laughter.\n\nThe scene shifts back to the present, with Claire expressing gratitude for the universe for bringing them together. Alessandra, a relationship expert, observes their interaction and comments on the relaxed atmosphere between Claire and Jesse, suggesting they are no longer at war with each other and can be themselves.\n\nJesse is then shown walking down a hallway at night, mentioning his plan to visit Janelle and Adam. The video then cuts to a sc

In [112]:
x=df[ df['asset_id']==('vlt_video_extract_SIXTY_MINUTES_60MI23_12_A_HBB.mp4')] 

In [ ]:
x['description'].to_list()[0]

'Here is a detailed description of the video you provided:\n\n**Category**\nNews\n\n**DetailedDescriptionOfEventsAndConversations**\nThe video opens with a dashcam video of a Porsche speeding on a freeway. The narrator says, "A speeding Porsche, a drugged truck driver." The scene shifts to a traffic camera view of a semi-truck driving on the freeway. Then, a man in a white forensic suit is seen in a car, his face partially obscured by shadows. An aerial shot reveals a multi-vehicle accident scene on the freeway at night, with emergency vehicles and personnel present. A man in handcuffs and a gray sweatshirt with a tennis racket design is escorted out of a brick building by a man in a suit. Reporters with microphones and cameras are gathered outside.\n\nThe scene transitions to photographs of four police officers in uniform. The narrator states, "Four officers killed." The scene returns to the accident, showing a tow truck next to the damaged semi-truck and Porsche. The man in the foren